<a href="https://colab.research.google.com/github/PROJET-TUTEURE-M2/model_relatives_psr_fr/blob/main/src/load_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/amina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import pandas as pd

# Charger le jeu de données

In [3]:
from nltk import RegexpTokenizer

def tokenizer(text):
  return RegexpTokenizer(r'''\w'|\w+|[^\w\s]''').tokenize(text)


In [4]:
import re

def replace_characters(match: re.Match) -> str:
    char = match.group(0)
    replacements = {'’': "'",'´': "'",'`': "'",'‘': "'",'«': '"','»': '"','“': '"','”': '"','–': '-','—': '-','…': ' ',u'\xa0': ' ',}
    return replacements[char]

def normalize_text(text: str) -> str:
    pattern = r'[’´`‘«»“”–—…]'
    return re.sub(pattern, replace_characters, text).strip()

In [5]:
def annotate(text, psr, tag):
    # Initialize the annotation list with 0s
    annotation = [0] * len(text)

    # Define the labels
    labels = {'O': 0, 'det': 1, 'appo': 2, 'ambiguë': 3}

    # Find the start index of the subsequence psr in text
    psr_len = len(psr)
    text_len = len(text)

    for i in range(text_len - psr_len + 1):
        if text[i:i + psr_len] == psr:
            annotation[i] = labels[tag]
            break

    return annotation

In [6]:
text = "cette frénésie, qui a touché tous les camps depuis tant de mois et parfois tant d'années, est indigne de nous et des principes de la République."
psr = "qui a touché tous les camps depuis tant de mois et parfois tant d'années"
tag = 'appo'
annotate(tokenizer(text), tokenizer(psr),tag)

[0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [32]:
file_path = '../data/relatives.xlsx'
sheet_name = 'data'
df = pd.read_excel(file_path, sheet_name=sheet_name)

In [8]:
df.head()

,texts,PSR,annotation
0,Simone Veil occupe désormais une place particu...,qui ont défiguré notre continent et avili l’hu...,det
1,une force qui a fait d’elle une actrice déterm...,qui a fait d’elle une actrice déterminante de ...,det
2,Les heures que nous vivons sont de celles-là.,que nous vivons,det
3,Je veux aujourd’hui vous parler du mandat que ...,que le peuple nous a donné,det
4,des institutions que je veux changer,que je veux changer,det


In [33]:
df = df.map(normalize_text)

In [34]:
df.head()

,texts,PSR,annotation
0,Simone Veil occupe désormais une place particu...,qui ont défiguré notre continent et avili l'hu...,det
1,une force qui a fait d'elle une actrice déterm...,qui a fait d'elle une actrice déterminante de ...,det
2,Les heures que nous vivons sont de celles-là.,que nous vivons,det
3,Je veux aujourd'hui vous parler du mandat que ...,que le peuple nous a donné,det
4,des institutions que je veux changer,que je veux changer,det


In [35]:
# Application de la fonction tokenizer sur les colonnes Phrase et PSR
df['texts'] = df['texts'].apply(tokenizer)
df['PSR'] = df['PSR'].apply(tokenizer)

In [36]:
df.head()

,texts,PSR,annotation
0,"[Simone, Veil, occupe, désormais, une, place, ...","[qui, ont, défiguré, notre, continent, et, avi...",det
1,"[une, force, qui, a, fait, d', elle, une, actr...","[qui, a, fait, d', elle, une, actrice, détermi...",det
2,"[Les, heures, que, nous, vivons, sont, de, cel...","[que, nous, vivons]",det
3,"[Je, veux, aujourd, ', hui, vous, parler, du, ...","[que, le, peuple, nous, a, donné]",det
4,"[des, institutions, que, je, veux, changer]","[que, je, veux, changer]",det


In [37]:
# Appliquer la fonction annotate à chaque ligne du DataFrame pour créer la colonne psr_tags
df['psr_tags'] = df.apply(lambda row: annotate(row['texts'], row['PSR'], row['annotation']), axis=1)

In [38]:
df.head()

,texts,PSR,annotation,psr_tags
0,"[Simone, Veil, occupe, désormais, une, place, ...","[qui, ont, défiguré, notre, continent, et, avi...",det,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[une, force, qui, a, fait, d', elle, une, actr...","[qui, a, fait, d', elle, une, actrice, détermi...",det,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[Les, heures, que, nous, vivons, sont, de, cel...","[que, nous, vivons]",det,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"[Je, veux, aujourd, ', hui, vous, parler, du, ...","[que, le, peuple, nous, a, donné]",det,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
4,"[des, institutions, que, je, veux, changer]","[que, je, veux, changer]",det,"[0, 0, 1, 0, 0, 0]"


In [39]:
df.drop(columns=['PSR','annotation'], inplace=True)
# Renommer la colonne Phrase en tokens
df.rename(columns={'texts': 'tokens'}, inplace=True)
df.reset_index(names='id' ,inplace=True)


In [40]:

df.head()

,id,tokens,psr_tags
0,0,"[Simone, Veil, occupe, désormais, une, place, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,"[une, force, qui, a, fait, d', elle, une, actr...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,"[Les, heures, que, nous, vivons, sont, de, cel...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
3,3,"[Je, veux, aujourd, ', hui, vous, parler, du, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
4,4,"[des, institutions, que, je, veux, changer]","[0, 0, 1, 0, 0, 0]"


# Créer un Dataset Hugging Face

## découpage du jeu de données en train, validation et test

In [41]:
#split the dataset into train , validation and test
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [46]:
# Supprimer les colonnes d'index auto-générées si elles existent
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [47]:
train_df.head()

,id,tokens,psr_tags
0,197,"[Mes, chers, collègues, ,, scellons, donc, auj...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,330,"[La, loi, du, 26, janvier, 2016, de, modernisa...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,321,"[Nous, avons, par, ailleurs, pris, différentes...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,430,"[Ces, travaux, débutent, aujourd, ', hui, sous...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,173,"[Ceux, qui, profitent, du, rouleau, compresseu...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


## Création d'un Dataset Hugging Face

In [43]:
from datasets import Dataset, Features, Sequence, Value, ClassLabel

In [48]:
# Définir les Features pour chaque colonne du DataFrame
features = Features({
    'id': Value(dtype='int32', id=None),
    'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
    'psr_tags': Sequence(feature=ClassLabel(names=['O', 'DET', 'APPO', 'AMBIGUE'], id=None), length=-1, id=None)
})

In [49]:
# Convertir le DataFrame en Dataset Hugging Face avec les Features spécifiés
train_ds = Dataset.from_pandas(train_df, features=features)
val_ds = Dataset.from_pandas(val_df, features=features)
test_ds = Dataset.from_pandas(test_df, features=features)

In [50]:
train_ds

Dataset({
    features: ['id', 'tokens', 'psr_tags'],
    num_rows: 281
})

In [51]:
from datasets import DatasetDict

# Recréation du DatasetDict
dataset = DatasetDict({
    'train': train_ds,
    'validation': val_ds,
    'test': test_ds
})

In [52]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'psr_tags'],
        num_rows: 281
    })
    validation: Dataset({
        features: ['id', 'tokens', 'psr_tags'],
        num_rows: 71
    })
    test: Dataset({
        features: ['id', 'tokens', 'psr_tags'],
        num_rows: 88
    })
})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Enregistrer le DatasetDict dans le hub Hugging Face
dataset.push_to_hub('relatives_psr_fr')